In [1]:
# Standard, ML, and Finance Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import yfinance as yf
import os
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from datetime import date, timedelta
import joblib
import warnings
import json
import optuna

warnings.filterwarnings('ignore')
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Set up device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"Versión de PyTorch: {torch.__version__}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

Usando dispositivo: cuda
Versión de PyTorch: 2.5.1
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


### 1. Pipeline Avanzado para el Mercado Mexicano (BMV)

Este pipeline está adaptado para analizar acciones del mercado mexicano. Integra optimización de hiperparámetros con Optuna y el guardado de artefactos para una eventual puesta en producción.

1.  **Definición del Portafolio y Datos Externos**: Se define una lista de acciones de la BMV (`TICKERS_A_ANALIZAR`) y tickers macroeconómicos relevantes para México (`MACRO_TICKERS`), como el tipo de cambio `USD/MXN` y el índice `IPC`.
2.  **Ingesta de Datos Completa**: Se descargan datos de mercado (OHLC), macroeconómicos y fundamentales para los tickers mexicanos.
3.  **Ingeniería de Características Robusta**: Se calculan múltiples indicadores técnicos (SMA, EMA, RSI, BBands, MACD, etc.) para cada acción objetivo.
4.  **Optimización de Hiperparámetros con Optuna**: Para cada acción, se ejecuta una búsqueda para encontrar la combinación óptima de `learning rate`, `hidden_size`, `num_layers` y `dropout` que minimice el error de validación, seleccionando incluso entre dos arquitecturas de modelo (Simple y Stacked LSTM).
5.  **Entrenamiento de Modelos Dedicados y Optimizados**: Se entrena un modelo LSTM final utilizando los mejores hiperparámetros encontrados. Esto asegura que cada modelo esté afinado para la dinámica de su activo específico.
6.  **Pronóstico y Recomendación**: Se generan pronósticos y se consolida una tabla de recomendaciones.
7.  **Guardado de Artefactos para Producción**: Al final del proceso, para la acción con el mayor potencial de ganancia a 30 días, se guardan todos los artefactos necesarios (`modelo`, `escalador`, `columnas`, `parámetros`) para poder cargar y utilizar el modelo en el futuro sin reentrenar.

In [2]:
# --- CONFIGURACIÓN DEL ANÁLISIS PARA EL MERCADO MEXICANO ---

# ## CORRECCIÓN CLAVE ##
# Se actualizó la lista a los componentes principales y verificados del S&P/BMV IPC para evitar errores de descarga.
TICKERS_A_ANALIZAR = [
    'WALMEX.MX', 'AMX.MX', 'GFNORTEO.MX', 'FEMSAUBD.MX', 'GMEXICOB.MX', 
    'CEMEXCPO.MX', 'GFINBURO.MX', 'BIMBOA.MX', 'ELEKTRA.MX', 'TLEVISACPO.MX',
    'GRUMAB.MX', 'GAPB.MX', 'ASURB.MX', 'ORBIA.MX', 'PINFRA.MX', 'AC.MX',
    'KOFUBL.MX', 'PEOLES.MX', 'ALPEKA.MX', 'KIMBERA.MX', 'BOLSAA.MX', 'LABB.MX',
    'GENTERA.MX', 'MEGACPO.MX', 'LIVEPOLC-1.MX', 'BBAJIOO.MX', 'GCARSOA1.MX',
    'RA.MX', 'CUERVO.MX', 'ALFAA.MX', 'VESTA.MX', 'GMXT.MX'
]

# Tickers para datos macroeconómicos relevantes para México
# ^MXX: Índice de Precios y Cotizaciones (IPC)
# MXN=X: Tipo de cambio USD a MXN
MACRO_TICKERS = ['^MXX', 'MXN=X']

# Parámetros del pipeline
WINDOW_SIZE = 90
N_FORECAST_DAYS = 30
N_OPTUNA_TRIALS = 15 # Aumentado ligeramente para una mejor búsqueda

# Directorio para guardar los artefactos del mejor modelo
ARTIFACTS_DIR = "production_artifacts_mx"
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# Fechas para la descarga de datos
start_date = (date.today() - timedelta(days=5*365)).strftime('%Y-%m-%d')
end_date = date.today().strftime('%Y-%m-%d')

In [3]:
# --- DEFINICIÓN DE FUNCIONES Y MODELOS ---

def add_technical_indicators(df, high_prices, low_prices, close_prices, ticker, window_size=20):
    """Añade indicadores técnicos al dataframe para un ticker específico."""
    # Asegurarse de que el ticker existe en los dataframes de precios
    if ticker not in close_prices.columns or ticker not in high_prices.columns or ticker not in low_prices.columns:
        return df
        
    df[f'{ticker}_SMA'] = close_prices[ticker].rolling(window=window_size).mean()
    df[f'{ticker}_EMA'] = close_prices[ticker].ewm(span=window_size, adjust=False).mean()
    delta = close_prices[ticker].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window_size).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window_size).mean()
    rs = gain / loss
    df[f'{ticker}_RSI'] = 100 - (100 / (1 + rs))
    bb_ma = close_prices[ticker].rolling(window=window_size).mean()
    bb_std = close_prices[ticker].rolling(window=window_size).std()
    df[f'{ticker}_BB_UPPER'] = bb_ma + (bb_std * 2)
    df[f'{ticker}_BB_LOWER'] = bb_ma - (bb_std * 2)
    ema_12 = close_prices[ticker].ewm(span=12, adjust=False).mean()
    ema_26 = close_prices[ticker].ewm(span=26, adjust=False).mean()
    df[f'{ticker}_MACD'] = ema_12 - ema_26
    df[f'{ticker}_MACD_signal'] = df[f'{ticker}_MACD'].ewm(span=9, adjust=False).mean()
    low_14 = low_prices[ticker].rolling(window=14).min()
    high_14 = high_prices[ticker].rolling(window=14).max()
    df[f'{ticker}_STOCH_K'] = 100 * ((close_prices[ticker] - low_14) / (high_14 - low_14))
    tr1 = high_prices[ticker] - low_prices[ticker]
    tr2 = np.abs(high_prices[ticker] - close_prices[ticker].shift())
    tr3 = np.abs(low_prices[ticker] - close_prices[ticker].shift())
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    df[f'{ticker}_ATR'] = true_range.ewm(alpha=1/14, adjust=False).mean()
    return df

def create_multivariate_sequences(data: np.ndarray, window_size: int):
    x, y = [], []
    for i in range(len(data) - window_size):
        x.append(data[i:(i + window_size), :])
        y.append(data[i + window_size, 0])
    return np.array(x), np.array(y)

def inverse_transform_preds(scaled_preds, scaler_obj, num_features):
    dummy = np.zeros((len(scaled_preds), num_features))
    dummy[:, 0] = np.array(scaled_preds).ravel()
    return scaler_obj.inverse_transform(dummy)[:, 0]

class SimpleLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=50):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)
    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :] 
        return self.linear(out).squeeze(-1)

class StackedLSTM(nn.Module):
    def __init__(self, input_size, hidden_size=50, num_layers=2, dropout=0.2):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0 
        )
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = lstm_out[:, -1, :]
        out = self.dropout(out)
        return self.linear(out).squeeze(-1)

In [4]:
# --- INGESTA DE DATOS (MERCADO, MACRO, FUNDAMENTALES) ---

ALL_TICKERS_TO_DOWNLOAD = TICKERS_A_ANALIZAR + MACRO_TICKERS
print(f"Descargando datos históricos (OHLC) de {start_date} a {end_date} para {len(ALL_TICKERS_TO_DOWNLOAD)} tickers...")

# --- Etapa 1: Descargar cada ticker individualmente para aislar fallos ---
data_dict = {}
successful_downloads = []
failed_downloads = []

for ticker in ALL_TICKERS_TO_DOWNLOAD:
    try:
        # Usamos auto_adjust=False para obtener el 'Adj Close' original si es necesario,
        # pero para este pipeline, 'Close' post-descarga individual es suficiente.
        data = yf.download(ticker, start=start_date, end=end_date, progress=False, auto_adjust=True)
        if data.empty:
            raise ValueError("No data returned for ticker.")
        data_dict[ticker] = data
        successful_downloads.append(ticker)
        print(f"  [OK] {ticker}")
    except Exception as e:
        failed_downloads.append(ticker)
        print(f"  [FAIL] {ticker}")

print("\nDescarga completada.")

# --- Etapa 2: Construir el DataFrame y extraer precios correctamente ---
if not data_dict:
    raise SystemExit("Error: No data was downloaded for any ticker. Halting execution.")

full_data = pd.concat(data_dict, axis=1)
full_data.interpolate(method='linear', limit_direction='both', inplace=True)

# ## CORRECCIÓN DEFINITIVA: Usar .xs() para acceder a datos en un MultiIndex ##
try:
    close_prices = full_data.xs('Close', level=1, axis=1)
    high_prices = full_data.xs('High', level=1, axis=1)
    low_prices = full_data.xs('Low', level=1, axis=1)
except KeyError:
    raise SystemExit("Error: Could not find 'Open', 'High', 'Low', 'Close' columns. Data download may have failed completely.")


# --- Etapa 3: Filtrar tickers por correlación para reducir multicolinealidad ---
DOWNLOADED_STOCK_TICKERS = [t for t in TICKERS_A_ANALIZAR if t in successful_downloads]
failed_stock_tickers = set(TICKERS_A_ANALIZAR) - set(DOWNLOADED_STOCK_TICKERS)

print(f"\nTickers de acciones que fallaron en la descarga inicial: {failed_stock_tickers}")
print(f"Se encontraron {len(DOWNLOADED_STOCK_TICKERS)} tickers de acciones para análisis de correlación.")

CORR_THRESHOLD = 0.95
columns_to_remove = set()

if len(DOWNLOADED_STOCK_TICKERS) > 1:
    print(f"\n--- Aplicando filtro de correlación (umbral > {CORR_THRESHOLD}) ---")
    # Asegurarse de que solo las columnas existentes se usan para la correlación
    existing_tickers_for_corr = [t for t in DOWNLOADED_STOCK_TICKERS if t in close_prices.columns]
    correlation_matrix = close_prices[existing_tickers_for_corr].corr()
    
    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > CORR_THRESHOLD:
                col_to_remove = correlation_matrix.columns[i]
                columns_to_remove.add(col_to_remove)
    
    if columns_to_remove:
        print(f"Tickers eliminados por alta correlación: {sorted(list(columns_to_remove))}")
    else:
        print("No se encontraron tickers con correlación superior al umbral.")
else:
    print("No hay suficientes tickers para el análisis de correlación. Saltando este paso.")

# La lista final de tickers a procesar es la lista descargada MENOS los altamente correlacionados
VALID_TICKERS = [t for t in DOWNLOADED_STOCK_TICKERS if t not in columns_to_remove]

print(f"\nSe procesarán {len(VALID_TICKERS)} tickers válidos después del filtrado.\n")


# --- Etapa 4: Obtener datos fundamentales para la lista final y válida ---
print("Obteniendo datos fundamentales (P/E, P/B) para los tickers válidos...")
fundamental_data = {}
if VALID_TICKERS:
    for ticker in VALID_TICKERS:
        try:
            info = yf.Ticker(ticker).info
            pe = info.get('trailingPE', np.nan)
            pb = info.get('priceToBook', np.nan)
            fundamental_data[ticker] = {'PE': pe, 'PB': pb}
        except Exception:
            fundamental_data[ticker] = {'PE': np.nan, 'PB': np.nan}
    fund_df = pd.DataFrame(fundamental_data).T.fillna(0)
else:
    fund_df = pd.DataFrame()

print("Datos fundamentales obtenidos.")

Descargando datos históricos (OHLC) de 2020-07-25 a 2025-07-24 para 34 tickers...
  [OK] WALMEX.MX



1 Failed download:
['AMX.MX']: YFTzMissingError('possibly delisted; no timezone found')


  [FAIL] AMX.MX
  [OK] GFNORTEO.MX
  [OK] FEMSAUBD.MX
  [OK] GMEXICOB.MX
  [OK] CEMEXCPO.MX
  [OK] GFINBURO.MX
  [OK] BIMBOA.MX
  [OK] ELEKTRA.MX
  [OK] TLEVISACPO.MX
  [OK] GRUMAB.MX
  [OK] GAPB.MX
  [OK] ASURB.MX
  [OK] ORBIA.MX
  [OK] PINFRA.MX
  [OK] AC.MX
  [OK] KOFUBL.MX


HTTP Error 404: 

1 Failed download:
['PEOLES.MX']: YFTzMissingError('possibly delisted; no timezone found')


  [FAIL] PEOLES.MX
  [OK] ALPEKA.MX
  [OK] KIMBERA.MX
  [OK] BOLSAA.MX
  [OK] LABB.MX
  [OK] GENTERA.MX
  [OK] MEGACPO.MX
  [OK] LIVEPOLC-1.MX
  [OK] BBAJIOO.MX
  [OK] GCARSOA1.MX
  [OK] RA.MX
  [OK] CUERVO.MX
  [OK] ALFAA.MX
  [OK] VESTA.MX
  [OK] GMXT.MX
  [OK] ^MXX
  [OK] MXN=X

Descarga completada.

Tickers de acciones que fallaron en la descarga inicial: {'PEOLES.MX', 'AMX.MX'}
Se encontraron 30 tickers de acciones para análisis de correlación.

--- Aplicando filtro de correlación (umbral > 0.95) ---
Tickers eliminados por alta correlación: [('KOFUBL.MX', 'KOFUBL.MX'), ('RA.MX', 'RA.MX')]

Se procesarán 30 tickers válidos después del filtrado.

Obteniendo datos fundamentales (P/E, P/B) para los tickers válidos...
Datos fundamentales obtenidos.


### 2. Búsqueda de Hiperparámetros y Entrenamiento

Aquí comienza el bucle principal. Para cada ticker de la BMV, se realizan los siguientes pasos:
1.  **Preparación de Datos**: Se ensambla el DataFrame de características, incluyendo precios de otras acciones mexicanas, datos macro (IPC, USD/MXN), fundamentales e indicadores técnicos.
2.  **Optimización (Optuna)**: Se define una función `objective` que Optuna usará para probar diferentes arquitecturas de modelo. Optuna buscará minimizar la pérdida de validación a lo largo de `N_OPTUNA_TRIALS`.
3.  **Entrenamiento Final**: Se instancia y entrena un nuevo modelo con los mejores hiperparámetros encontrados.
4.  **Pronóstico**: Se utiliza el modelo final optimizado para predecir los precios futuros.
5.  **Almacenamiento de Artefactos**: Los artefactos (modelo, escalador, columnas) del modelo entrenado se guardan temporalmente en un diccionario.

In [10]:
all_recommendations = []
all_forecasts = {}
all_artifacts = {}

# ## Iterar sobre la lista de tickers VALIDOS ##
for target_ticker in VALID_TICKERS:
    print(f"\n{'='*20} PROCESANDO: {target_ticker} {'='*20}")
    
    try:
        # Robustly get the last actual price as a scalar.
        price_series = close_prices[target_ticker].dropna()
        if price_series.empty:
            print(f"No hay datos de precios válidos para {target_ticker}. Saltando...")
            continue
        
        last_actual_price = price_series.iloc[-1]
        if isinstance(last_actual_price, (pd.Series, pd.DataFrame)):
            last_actual_price = last_actual_price.iloc[0]

        if pd.isna(last_actual_price) or last_actual_price <= 0:
            print(f"Precio actual inválido o no encontrado para {target_ticker} [Valor: {last_actual_price}]. Saltando...")
            continue

        # 1. Preparación de Datos
        feature_stock_cols = [c for c in VALID_TICKERS if c != target_ticker and c in close_prices.columns]
        macro_cols = [c for c in MACRO_TICKERS if c in close_prices.columns]
        ordered_cols = [target_ticker] + feature_stock_cols + macro_cols
        
        data_for_ticker = close_prices[ordered_cols].copy()
        data_for_ticker.dropna(how='all', axis=1, inplace=True)

        features_df = data_for_ticker.copy()
        features_df = add_technical_indicators(features_df, high_prices, low_prices, close_prices, target_ticker)
        
        if target_ticker in fund_df.index:
            features_df[f'{target_ticker}_PE'] = fund_df.loc[target_ticker, 'PE']
            features_df[f'{target_ticker}_PB'] = fund_df.loc[target_ticker, 'PB']
        else:
            features_df[f'{target_ticker}_PE'] = 0
            features_df[f'{target_ticker}_PB'] = 0
            
        features_df.fillna(method='ffill', inplace=True)
        features_df.dropna(inplace=True)

        if len(features_df) < WINDOW_SIZE * 2: 
            print(f"No hay suficientes datos para {target_ticker} después de la preparación. Saltando...")
            continue
        
        # --- MODIFICACIÓN: Preparación para Time Series Cross-Validation ---
        # No se realiza un train/test split aquí, se hará dentro de la validación cruzada.
        feature_scaler = MinMaxScaler()
        features_scaled = feature_scaler.fit_transform(features_df)
        X, y = create_multivariate_sequences(features_scaled, WINDOW_SIZE)
        
        if len(X) < 100: # Asegurar suficientes datos para la validación
            print(f"Datos insuficientes para la validación cruzada en {target_ticker}. Saltando...")
            continue

        NUM_FEATURES = X.shape[2]
        
        # Configurar el TimeSeriesSplit
        N_SPLITS = 5  # Número de folds para la validación cruzada
        tscv = TimeSeriesSplit(n_splits=N_SPLITS)

        tracker = {
            'best_loss': float('inf'),
            'best_model_state_dict': None,
            'best_model_params': None
        }

        # --- MODIFICACIÓN: `objective` con Time Series Cross-Validation ---
        def objective(trial):
            model_type = trial.suggest_categorical('model_type', ['SimpleLSTM', 'StackedLSTM'])
            params = {
                'model_type': model_type, 'input_size': NUM_FEATURES,
                'lr': trial.suggest_float('lr', 1e-4, 1e-2, log=True),
                'hidden_size': trial.suggest_int('hidden_size', 32, 128)
            }
            if model_type == 'StackedLSTM':
                params['num_layers'] = trial.suggest_int('num_layers', 1, 2)
                params['dropout'] = trial.suggest_float('dropout', 0.1, 0.5)
            
            # Almacenar el error de cada fold
            fold_errors = []
            
            # Iterar sobre los splits de la validación cruzada
            for train_index, val_index in tscv.split(X):
                X_train, X_val = X[train_index], X[val_index]
                y_train, y_val = y[train_index], y[val_index]

                X_train_t, y_train_t = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
                X_val_t, y_val_t = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)
                
                train_loader = DataLoader(TensorDataset(X_train_t, y_train_t), batch_size=64, shuffle=True)
                val_loader = DataLoader(TensorDataset(X_val_t, y_val_t), batch_size=64, shuffle=False)
                
                # Instanciar el modelo para este fold
                if model_type == 'StackedLSTM':
                    model = StackedLSTM(params['input_size'], params['hidden_size'], params['num_layers'], params['dropout']).to(device)
                else:
                    model = SimpleLSTM(params['input_size'], params['hidden_size']).to(device)

                optimizer = optim.Adam(model.parameters(), lr=params['lr'])
                criterion = nn.MSELoss()
                
                # Entrenar el modelo para este fold
                for epoch in range(50): # Un número fijo de épocas por fold es suficiente
                    model.train()
                    for X_b, y_b in train_loader: X_b, y_b = X_b.to(device), y_b.to(device); optimizer.zero_grad(); loss = criterion(model(X_b), y_b); loss.backward(); optimizer.step()
                
                # Evaluar en el conjunto de validación del fold
                model.eval(); val_loss = 0
                with torch.no_grad():
                    for X_b, y_b in val_loader: X_b, y_b = X_b.to(device), y_b.to(device); val_loss += criterion(model(X_b), y_b).item()
                
                fold_errors.append(val_loss / len(val_loader))

            # El valor a minimizar es el promedio de los errores de todos los folds
            avg_cv_error = np.mean(fold_errors)
            
            # Guardar el mejor modelo basado en el error de la CV
            if avg_cv_error < tracker['best_loss']:
                tracker['best_loss'] = avg_cv_error
                # No podemos guardar un state_dict aquí, ya que el modelo se reentrena en cada fold.
                # Solo guardamos los parámetros. El modelo final se reentrenará con todos los datos.
                tracker['best_model_params'] = params
                
            return avg_cv_error

        print(f"Iniciando búsqueda de hiperparámetros con Time Series Cross-Validation para {target_ticker}...")
        study = optuna.create_study(direction='minimize')
        study.optimize(objective, n_trials=N_OPTUNA_TRIALS, timeout=600) # Aumentar timeout para CV
        
        if not tracker['best_model_params']:
            print(f"Optuna no encontró un modelo válido para {target_ticker}. Saltando...")
            continue
            
        best_model_params = tracker['best_model_params'].copy()
        display_params = {k: v for k, v in best_model_params.items() if k not in ['input_size']}
        print(f"Búsqueda completada. Mejor Arquitectura y Parámetros: {display_params}")

        # --- MODIFICACIÓN: Reentrenar el mejor modelo con TODOS los datos disponibles ---
        print(f"Reentrenando el mejor modelo ({best_model_params['model_type']}) con todos los datos...")
        final_model_type = best_model_params.pop('model_type')
        final_input_size = best_model_params.pop('input_size')
        final_lr = best_model_params.pop('lr')

        if final_model_type == 'SimpleLSTM': model = SimpleLSTM(input_size=final_input_size, **best_model_params).to(device)
        else: model = StackedLSTM(input_size=final_input_size, **best_model_params).to(device)
        
        # Preparar datos para reentrenamiento final
        X_all_t = torch.tensor(X, dtype=torch.float32)
        y_all_t = torch.tensor(y, dtype=torch.float32)
        final_train_loader = DataLoader(TensorDataset(X_all_t, y_all_t), batch_size=64, shuffle=True)
        optimizer = optim.Adam(model.parameters(), lr=final_lr)
        criterion = nn.MSELoss()

        # Reentrenamiento final
        for epoch in range(70): # Más épocas para el entrenamiento final
            model.train()
            for X_b, y_b in final_train_loader: X_b, y_b = X_b.to(device), y_b.to(device); optimizer.zero_grad(); loss = criterion(model(X_b), y_b); loss.backward(); optimizer.step()

        all_artifacts[target_ticker] = {
            'model_state': model.state_dict(), 'scaler': feature_scaler,
            'columns': features_df.columns.tolist(),
            'model_params': {'model_type': final_model_type, 'input_size': final_input_size, **best_model_params}
        }

        print(f"Generando pronóstico para {target_ticker}...")
        last_sequence = features_scaled[-WINDOW_SIZE:]
        current_sequence_t = torch.tensor(last_sequence, dtype=torch.float32).unsqueeze(0).to(device)
        future_preds_scaled = []
        model.eval()
        with torch.no_grad():
            for _ in range(N_FORECAST_DAYS):
                next_pred = model(current_sequence_t)
                future_preds_scaled.append(next_pred.item())
                new_step = current_sequence_t[0, -1, :].clone(); new_step[0] = next_pred.item()
                current_sequence_t = torch.cat((current_sequence_t[:, 1:, :], new_step.unsqueeze(0).unsqueeze(1)), dim=1)

        future_preds_inv = inverse_transform_preds(future_preds_scaled, feature_scaler, NUM_FEATURES)
        
        horizons = {'Mañana (1 día)': 1, 'Próxima Semana (7 días)': 7, 'Próximo Mes (30 días)': N_FORECAST_DAYS}
        ticker_recs = {"Ticker": target_ticker, "Precio Actual (MXN)": last_actual_price}
        for name, day in horizons.items():
            if day <= len(future_preds_inv):
                pred_price = future_preds_inv[day-1]
                change_pct = (pred_price / last_actual_price - 1) * 100
            else:
                pred_price = float('nan'); change_pct = float('nan')
            ticker_recs[f'Predicción {name}'] = pred_price
            ticker_recs[f'Cambio % {name}'] = change_pct
        all_recommendations.append(ticker_recs)
        
        last_date = features_df.index[-1]
        future_dates = pd.to_datetime([last_date + timedelta(days=i) for i in range(1, N_FORECAST_DAYS + 1)])
        all_forecasts[target_ticker] = {'history': features_df.iloc[-90:][target_ticker], 'forecast_dates': future_dates, 'forecast_values': future_preds_inv}
        all_artifacts[target_ticker]['features_df'] = features_df # Guardamos el df con todos los indicadores

    except Exception as e:
        import traceback; print(f"ERROR procesando {target_ticker}: {e}"); traceback.print_exc()

print(f"\n{'='*20} ANÁLISIS COMPLETADO {'='*20}")


==================== PROCESANDO: WALMEX.MX ====================
Iniciando búsqueda de hiperparámetros con Time Series Cross-Validation para WALMEX.MX...
Búsqueda completada. Mejor Arquitectura y Parámetros: {'model_type': 'SimpleLSTM', 'lr': 0.0036263645855953946, 'hidden_size': 104}
Reentrenando el mejor modelo (SimpleLSTM) con todos los datos...
Generando pronóstico para WALMEX.MX...

==================== PROCESANDO: GFNORTEO.MX ====================
Iniciando búsqueda de hiperparámetros con Time Series Cross-Validation para GFNORTEO.MX...
Búsqueda completada. Mejor Arquitectura y Parámetros: {'model_type': 'SimpleLSTM', 'lr': 0.0023219400312358227, 'hidden_size': 101}
Reentrenando el mejor modelo (SimpleLSTM) con todos los datos...
Generando pronóstico para GFNORTEO.MX...

==================== PROCESANDO: FEMSAUBD.MX ====================
Iniciando búsqueda de hiperparámetros con Time Series Cross-Validation para FEMSAUBD.MX...
Búsqueda completada. Mejor Arquitectura y Parámetros: {'

### 3. Tabla de Resultados y Recomendaciones

La siguiente tabla consolida los resultados del mercado mexicano, mostrando el rendimiento esperado para cada acción en diferentes horizontes de tiempo. Las celdas se colorean según el potencial de ganancia (verde) o pérdida (rojo).

In [12]:
# --- PRESENTACIÓN DE RESULTADOS ---

if all_recommendations:
    results_df = pd.DataFrame(all_recommendations).set_index("Ticker")

    # --- Diagnóstico: Mostrar los tipos de datos ANTES de la corrección ---
    print("--- Tipos de datos en el DataFrame ANTES de la corrección: ---")
    print(results_df.info())
    print("-" * 60)

    # ## CORRECCIÓN DEFINITIVA: Forzar la conversión de columnas a tipo numérico ##
    # Identificar las columnas que deberían ser números
    numeric_cols = [
        'Precio Actual (MXN)', 'Predicción Mañana (1 día)', 'Cambio % Mañana (1 día)',
        'Predicción Próxima Semana (7 días)', 'Cambio % Próxima Semana (7 días)',
        'Predicción Próximo Mes (30 días)', 'Cambio % Próximo Mes (30 días)'
    ]
    
    # Aplicar la conversión, los errores se convertirán en NaN
    for col in numeric_cols:
        if col in results_df.columns:
            results_df[col] = pd.to_numeric(results_df[col], errors='coerce')

    # --- Diagnóstico: Mostrar los tipos de datos DESPUÉS de la corrección ---
    print("--- Tipos de datos en el DataFrame DESPUÉS de la corrección: ---")
    print(results_df.info())
    print("-" * 60)


    # Ahora que los datos están limpios (solo números o NaN), el Styler funcionará
    styled_df = results_df.style.format({
        'Precio Actual (MXN)': "${:,.2f}",
        'Predicción Mañana (1 día)': "${:,.2f}", 
        'Cambio % Mañana (1 día)': "{:.2f}%",
        'Predicción Próxima Semana (7 días)': "${:,.2f}", 
        'Cambio % Próxima Semana (7 días)': "{:.2f}%",
        'Predicción Próximo Mes (30 días)': "${:,.2f}", 
        'Cambio % Próximo Mes (30 días)': "{:.2f}%"
    }, na_rep="-").background_gradient(cmap='RdYlGn', subset=numeric_cols) \
                   .set_caption("Pronóstico de Rendimiento por Acción (BMV)")
    
    print("Tabla de Resultados Consolidados para la BMV:")
    display(styled_df)
else:
    print("No se pudo generar ninguna recomendación. Los tickers válidos no produjeron resultados.")

--- Tipos de datos en el DataFrame ANTES de la corrección: ---
<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, WALMEX.MX to GMXT.MX
Data columns (total 7 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Precio Actual (MXN)                 30 non-null     float64
 1   Predicción Mañana (1 día)           30 non-null     float64
 2   Cambio % Mañana (1 día)             30 non-null     float64
 3   Predicción Próxima Semana (7 días)  30 non-null     float64
 4   Cambio % Próxima Semana (7 días)    30 non-null     float64
 5   Predicción Próximo Mes (30 días)    30 non-null     float64
 6   Cambio % Próximo Mes (30 días)      30 non-null     float64
dtypes: float64(7)
memory usage: 1.9+ KB
None
------------------------------------------------------------
--- Tipos de datos en el DataFrame DESPUÉS de la corrección: ---
<class 'pandas.core.frame.DataFrame'>
Index: 30 entries, WALMEX.MX to 

,Precio Actual (MXN),Predicción Mañana (1 día),Cambio % Mañana (1 día),Predicción Próxima Semana (7 días),Cambio % Próxima Semana (7 días),Predicción Próximo Mes (30 días),Cambio % Próximo Mes (30 días)
Ticker,,,,,,,
WALMEX.MX,$53.99,$53.42,-1.06%,$53.48,-0.94%,$53.51,-0.89%
GFNORTEO.MX,$165.32,$165.30,-0.01%,$163.89,-0.86%,$163.86,-0.88%
FEMSAUBD.MX,$180.71,$184.55,2.12%,$184.53,2.11%,$185.26,2.52%
GMEXICOB.MX,$119.24,$118.44,-0.67%,$118.37,-0.73%,$118.34,-0.76%
CEMEXCPO.MX,$14.78,$14.67,-0.77%,$14.56,-1.51%,$14.56,-1.52%
GFINBURO.MX,$49.97,$50.05,0.15%,$50.35,0.76%,$50.36,0.78%
BIMBOA.MX,$50.74,$49.31,-2.83%,$49.72,-2.01%,$49.39,-2.65%
ELEKTRA.MX,$369.02,$370.67,0.45%,$372.66,0.99%,$372.68,0.99%
TLEVISACPO.MX,$9.00,$8.63,-4.08%,$8.56,-4.85%,$8.56,-4.88%


In [20]:
# --- PANEL DE CONTROL DE LAS MEJores OPORTUNIDADES (VERSIÓN CONSOLIDADA) ---
from plotly.subplots import make_subplots
import numpy as np

# Verificar si se generaron resultados para poder graficar
if 'results_df' in locals() and not results_df.empty and 'all_forecasts' in locals():
    
    # Ordenar los resultados para encontrar los 3 mejores tickers
    change_col_30d = 'Cambio % Próximo Mes (30 días)'
    results_df[change_col_30d] = pd.to_numeric(results_df[change_col_30d], errors='coerce')
    top_3_tickers = results_df.sort_values(by=change_col_30d, ascending=False).head(3).index.tolist()

    if top_3_tickers:
        print(f"Generando panel de control consolidado para: {', '.join(top_3_tickers)}")

        num_tickers = len(top_3_tickers)
        
        # --- AJUSTE: Ahora son 3 filas por ticker y se ajustan las alturas ---
        fig = make_subplots(
            rows=num_tickers * 3, 
            cols=1, 
            shared_xaxes=True,
            # Títulos para cada grupo de gráficos
            subplot_titles=[
                f"<b>{ticker}</b> - Precio y Pronóstico" if i % 3 == 0 else "" 
                for i, ticker in enumerate(np.repeat(top_3_tickers, 3))
            ],
            row_heights=[0.5, 0.3, 0.2] * num_tickers, # Más espacio para el precio, menos para RSI
            vertical_spacing=0.05
        )
        
        for i, ticker in enumerate(top_3_tickers):
            if ticker in all_forecasts and ticker in all_artifacts and 'features_df' in all_artifacts[ticker]:
                forecast_data = all_forecasts[ticker]
                features_df = all_artifacts[ticker]['features_df']
                plot_df = features_df.iloc[-252:]  # Último año de datos para contexto
                
                start_row = i * 3 + 1
                show_legend_for_this_group = (i == 0) # Leyenda solo para el primer ticker

                # --- Gráfico 1: Historial y Pronóstico (Sin cambios) ---
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[ticker].values.ravel(), name='Historial', line=dict(color='#1f77b4'), showlegend=show_legend_for_this_group), row=start_row, col=1)
                fig.add_trace(go.Scatter(x=forecast_data['forecast_dates'], y=forecast_data['forecast_values'], name='Pronóstico', line=dict(color='#ff7f0e', dash='dash'), showlegend=show_legend_for_this_group), row=start_row, col=1)
                fig.update_yaxes(title_text="Precio", row=start_row, col=1)

                # Nombres de las columnas de indicadores
                sma_col, ema_col, rsi_col = f'{ticker}_SMA', f'{ticker}_EMA', f'{ticker}_RSI'
                bb_upper_col, bb_lower_col = f'{ticker}_BB_UPPER', f'{ticker}_BB_LOWER'

                # --- Gráfico 2: INDICADORES TÉCNICOS CONSOLIDADOS (SMA, EMA, BBands) ---
                # Se renderiza en este orden para que las capas se vean bien: Relleno -> Líneas -> Precio
                
                # 1. Bandas de Bollinger (área de fondo)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[bb_upper_col].values.ravel(), line=dict(color='rgba(0,0,0,0)'), showlegend=False), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[bb_lower_col].values.ravel(), name='B. Bollinger', line=dict(color='rgba(0,0,0,0)'), fill='tonexty', fillcolor='rgba(0,100,80,0.15)', showlegend=show_legend_for_this_group), row=start_row + 1, col=1)
                
                # 2. Medias Móviles
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[sma_col].values.ravel(), name='SMA', line=dict(color='blue', dash='dot', width=1.5), showlegend=show_legend_for_this_group), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[ema_col].values.ravel(), name='EMA', line=dict(color='orange', dash='dot', width=1.5), showlegend=show_legend_for_this_group), row=start_row + 1, col=1)
                
                # 3. Precio (encima de todo)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[ticker].values.ravel(), name='Precio', line=dict(color='black', width=1.2), showlegend=False), row=start_row + 1, col=1)
                fig.update_yaxes(title_text="Indicadores", row=start_row + 1, col=1)

                # --- Gráfico 3: RSI (Ahora en la fila start_row + 2) ---
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[rsi_col].values.ravel(), name='RSI', line=dict(color='purple'), showlegend=show_legend_for_this_group), row=start_row + 2, col=1)
                fig.add_hline(y=70, line_dash="dash", line_color="red", row=start_row + 2, col=1)
                fig.add_hline(y=30, line_dash="dash", line_color="green", row=start_row + 2, col=1)
                fig.update_yaxes(title_text="RSI", range=[0, 100], row=start_row + 2, col=1)
                
            else:
                 print(f"Advertencia: No se encontraron datos completos para {ticker}. Saltando su gráfico.")

        # Actualizar el diseño general de la figura
        fig.update_layout(
            height=300 * num_tickers * 3,  # Altura ajustada para 3 gráficos por ticker
            title_text='<b>Panel de Control de Oportunidades (Precio, Indicadores y RSI)</b>',
            template='plotly_white',
            hovermode='x unified',
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        
        # Actualizar el eje X final
        fig.update_xaxes(title_text="<b>Fecha</b>", row=num_tickers * 3, col=1)
        fig.show()
    else:
        print("No se encontraron suficientes tickers con pronósticos válidos para graficar.")
else:
    print("Los datos de resultados o pronósticos no están disponibles. Asegúrese de haber ejecutado las celdas anteriores.")

Generando panel de control consolidado para: KOFUBL.MX, CUERVO.MX, ORBIA.MX


In [19]:
# --- ANÁLISIS COMPARATIVO DE ESTRATEGIAS Y PANEL DE CONTROL ---
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

# --- 1. Función Auxiliar para el Pronóstico ARIMA ---
def get_arima_forecast(ticker_data, forecast_days):
    """Genera un pronóstico ARIMA para una serie de tiempo dada."""
    try:
        # Usar un modelo ARIMA simple (autorregresivo, diferenciado, media móvil)
        model = ARIMA(ticker_data, order=(5,1,0))
        model_fit = model.fit()
        return model_fit.forecast(steps=forecast_days)
    except Exception as e:
        # Si el modelo no converge, retorna un array de NaNs
        return np.full(forecast_days, np.nan)

# --- 2. Preparación de Datos y Tabla de Estrategias ---
if 'all_recommendations' in locals() and all_recommendations:
    results_df = pd.DataFrame(all_recommendations).set_index("Ticker")
    change_cols = [col for col in results_df.columns if 'Cambio %' in col]
    for col in change_cols:
        results_df[col] = pd.to_numeric(results_df[col], errors='coerce')

    horizons = {
        'Diario (1 día)': 1, 
        'Semanal (7 días)': 7, 
        'Quincenal (15 días)': 15, 
        'Mensual (30 días)': 30
    }
    
    strategy_results = []
    
    # Calcular rendimientos de ARIMA y Benchmark
    arima_returns = {}
    for ticker in VALID_TICKERS:
        history = close_prices[ticker].dropna()
        if len(history) > 30: # Necesita suficientes datos para el modelo
            forecast = get_arima_forecast(history, 30)
            returns = (forecast / history.iloc[-1] - 1) * 100
            arima_returns[ticker] = returns

    arima_returns_df = pd.DataFrame(arima_returns).T
    
    # Benchmark: Pronóstico del IPC (^MXX)
    ipc_history = close_prices['^MXX'].dropna()
    ipc_forecast = get_arima_forecast(ipc_history, 30)
    ipc_returns = (ipc_forecast / ipc_history.iloc[-1] - 1) * 100

    for name, days in horizons.items():
        # Estrategia LSTM
        lstm_col = f'Cambio % {name}'
        if lstm_col in results_df.columns:
            top_5_lstm = results_df[lstm_col].nlargest(5).mean()
        else:
            top_5_lstm = np.nan
        
        # Estrategia ARIMA
        if (days - 1) in arima_returns_df.columns:
            top_5_arima = arima_returns_df[days - 1].nlargest(5).mean()
        else:
            top_5_arima = np.nan
            
        # Benchmark del Mercado
        benchmark_return = ipc_returns.iloc[days - 1]
        
        strategy_results.append({
            'Horizonte': name,
            'Top 5 LSTM': top_5_lstm,
            'Top 5 ARIMA': top_5_arima,
            'Benchmark (IPC)': benchmark_return
        })
        
    strategy_df = pd.DataFrame(strategy_results).set_index('Horizonte')
    styled_strategy_df = strategy_df.style.format("{:+.2f}%").background_gradient(cmap='Greens').set_caption("<b>Rendimiento Esperado por Estrategia</b>")
    
    print("\n--- 🏆 TABLA COMPARATIVA DE ESTRATEGIAS 🏆 ---")
    display(styled_strategy_df)

    # --- 3. PANEL DE CONTROL COMPLETO PARA LOS 3 MEJORES TICKERS (según LSTM) ---
    print("\n--- 📊 GENERANDO PANEL DE CONTROL DE LAS MEJORES OPORTUNIDADES 📊 ---")
    change_col_30d = 'Cambio % Próximo Mes (30 días)'
    top_3_tickers_df = results_df.sort_values(by=change_col_30d, ascending=False).head(3)
    top_3_tickers = top_3_tickers_df.index.tolist()
    
    if top_3_tickers:
        num_tickers = len(top_3_tickers)
        fig = make_subplots(
            rows=num_tickers * 3, cols=1, shared_xaxes=True,
            row_heights=[0.5, 0.25, 0.25] * num_tickers,
            subplot_titles=[f"<b>{ticker}</b> (LSTM: {top_3_tickers_df.loc[ticker, change_col_30d]:+.2f}%)" if i%3==0 else "" for i, ticker in enumerate(np.repeat(top_3_tickers, 3))],
            vertical_spacing=0.04
        )
        
        for i, ticker in enumerate(top_3_tickers):
            if ticker in all_forecasts and ticker in all_artifacts and 'features_df' in all_artifacts[ticker]:
                forecast_data = all_forecasts[ticker]
                features_df_full = all_artifacts[ticker]['features_df']
                plot_df = features_df_full.iloc[-252:]
                
                start_row = i * 3 + 1
                show_legend = (i == 0)

                # --- Gráfico 1: Precio y Pronósticos ---
                history = plot_df[ticker]
                lstm_forecast_vals = forecast_data['forecast_values']
                arima_forecast_vals = get_arima_forecast(history, 30)

                fig.add_trace(go.Scatter(x=history.index, y=history.values.ravel(), name='Historial', line=dict(color='black')), row=start_row, col=1)
                fig.add_trace(go.Scatter(x=forecast_data['forecast_dates'], y=lstm_forecast_vals, name='Pronóstico LSTM', line=dict(color='red', dash='dash'), showlegend=show_legend), row=start_row, col=1)
                fig.add_trace(go.Scatter(x=forecast_data['forecast_dates'], y=arima_forecast_vals, name='Pronóstico ARIMA', line=dict(color='purple', dash='dot'), showlegend=show_legend), row=start_row, col=1)
                fig.update_yaxes(title_text="Precio", row=start_row, col=1)

                # --- Gráfico 2: Indicadores de Tendencia y Volatilidad (MA & BB) ---
                sma_col, ema_col, bb_upper_col, bb_lower_col = f'{ticker}_SMA', f'{ticker}_EMA', f'{ticker}_BB_UPPER', f'{ticker}_BB_LOWER'
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[bb_upper_col].values.ravel(), line=dict(color='rgba(0,0,0,0)'), showlegend=False), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[bb_lower_col].values.ravel(), name='Bandas Bollinger', fill='tonexty', fillcolor='rgba(0,100,80,0.2)', line=dict(color='rgba(0,0,0,0)'), showlegend=show_legend), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[ticker].values.ravel(), name='Precio', line=dict(color='black', width=0.7), showlegend=False), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[sma_col].values.ravel(), name='SMA', line=dict(color='blue', dash='dot'), showlegend=show_legend), row=start_row + 1, col=1)
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[ema_col].values.ravel(), name='EMA', line=dict(color='orange', dash='dot'), showlegend=show_legend), row=start_row + 1, col=1)
                fig.update_yaxes(title_text="Indicadores", row=start_row + 1, col=1)

                # --- Gráfico 3: RSI ---
                rsi_col = f'{ticker}_RSI'
                fig.add_trace(go.Scatter(x=plot_df.index, y=plot_df[rsi_col].values.ravel(), name='RSI', line=dict(color='purple'), showlegend=show_legend), row=start_row + 2, col=1)
                fig.add_hline(y=70, line_dash="dash", line_color="red", row=start_row + 2, col=1)
                fig.add_hline(y=30, line_dash="dash", line_color="green", row=start_row + 2, col=1)
                fig.add_annotation(x=plot_df.index[30], y=75, text="Sobrecompra", showarrow=False, row=start_row + 2, col=1)
                fig.add_annotation(x=plot_df.index[30], y=25, text="Sobrevendido", showarrow=False, row=start_row + 2, col=1)
                fig.update_yaxes(title_text="RSI", range=[0, 100], row=start_row + 2, col=1)
            else:
                 print(f"Advertencia: No se encontraron datos completos para {ticker}.")

        fig.update_layout(
            height=300 * num_tickers * 3, title_text='<b>Panel de Control de las Mejores Oportunidades Mensuales</b>',
            template='plotly_white', hovermode='x unified',
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
        )
        fig.update_xaxes(title_text="<b>Fecha</b>", row=num_tickers * 3, col=1)
        fig.show()

else:
    print("No se pudo generar ninguna recomendación o los resultados están vacíos.")

/home/nacho/.conda/envs/torch/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/nacho/.conda/envs/torch/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/nacho/.conda/envs/torch/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning:

A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.

/home/nacho/.conda/envs/torch/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:837: ValueWarning:

No supported index is available. Prediction results will be given with an integer index beginning at `start`.

/home/nacho/.conda/envs/torch/lib/python3.10/site-packages/

TypeError: '<' not supported between instances of 'str' and 'int'

### 4. Recomendaciones Finales y Guardado de Artefactos

Finalmente, se extraen las mejores oportunidades para cada horizonte de tiempo. Para la acción con el mejor pronóstico a 30 días, sus artefactos (modelo, escalador y columnas) se guardan en el disco. Esto permite que el modelo más prometedor sea fácilmente cargado y utilizado en otro entorno para realizar predicciones sobre nuevos datos sin necesidad de repetir todo el proceso de entrenamiento y optimización.

In [18]:
# --- GUARDADO DE ARTEFACTOS PARA EL MEJOR MODELO ---
if all_recommendations:
    results_df = pd.DataFrame(all_recommendations).set_index("Ticker")

    # Asegurarse de que la columna de % de cambio es numérica antes de encontrar el máximo
    change_col_30d = 'Cambio % Próximo Mes (30 días)'
    results_df[change_col_30d] = pd.to_numeric(results_df[change_col_30d], errors='coerce')
    results_df.dropna(subset=[change_col_30d], inplace=True)
    
    if not results_df.empty:
        best_ticker_30d = results_df[change_col_30d].idxmax()
        best_gain_30d = results_df.loc[best_ticker_30d, change_col_30d]

        print(f"\nMejor oportunidad a 30 días: {best_ticker_30d} con un rendimiento esperado de {best_gain_30d:.2f}%.")
        print(f"Guardando artefactos para {best_ticker_30d} en el directorio '{ARTIFACTS_DIR}'...")

        # Extraer los artefactos para el mejor ticker
        artifacts_to_save = all_artifacts.get(best_ticker_30d)

        if artifacts_to_save:
            # Guardar el modelo (state_dict)
            model_path = os.path.join(ARTIFACTS_DIR, f"{best_ticker_30d}_model.pth")
            torch.save(artifacts_to_save['model_state'], model_path)

            # Guardar el escalador
            scaler_path = os.path.join(ARTIFACTS_DIR, f"{best_ticker_30d}_scaler.joblib")
            joblib.dump(artifacts_to_save['scaler'], scaler_path)

            # Guardar metadatos (columnas y parámetros del modelo)
            metadata = {
                'columns': artifacts_to_save['columns'],
                'model_params': artifacts_to_save['model_params'],
                'window_size': WINDOW_SIZE
            }
            metadata_path = os.path.join(ARTIFACTS_DIR, f"{best_ticker_30d}_metadata.json")
            with open(metadata_path, 'w') as f:
                json.dump(metadata, f, indent=4)
            
            print("Artefactos guardados exitosamente:")
            print(f"  - Modelo: {model_path}")
            print(f"  - Escalador: {scaler_path}")
            print(f"  - Metadatos: {metadata_path}")
        else:
            print(f"Error: No se encontraron artefactos para {best_ticker_30d}.")
    else:
        print("No se encontraron resultados válidos para determinar la mejor oportunidad.")
else:
    print("No se generaron recomendaciones, no se guardarán artefactos.")



Mejor oportunidad a 30 días: KOFUBL.MX con un rendimiento esperado de 4.58%.
Guardando artefactos para KOFUBL.MX en el directorio 'production_artifacts_mx'...
Artefactos guardados exitosamente:
  - Modelo: production_artifacts_mx/KOFUBL.MX_model.pth
  - Escalador: production_artifacts_mx/KOFUBL.MX_scaler.joblib
  - Metadatos: production_artifacts_mx/KOFUBL.MX_metadata.json


### 5. Conclusión y Próximos Pasos

Este pipeline ha sido adaptado exitosamente para analizar el mercado de valores mexicano (BMV). Al automatizar no solo el análisis y la predicción, sino también la optimización del modelo y la preparación para su despliegue, hemos creado un sistema de extremo a extremo que es a la vez potente y práctico.

**Próximos Pasos Sugeridos:**
1.  **Script de Inferencia**: Crear un script de Python (`predict_mx.py`) que cargue los artefactos guardados (`_model.pth`, `_scaler.joblib`, etc.) para realizar una predicción para la acción seleccionada con nuevos datos, sin necesidad de reentrenamiento. 
2.  **Análisis de Sentimiento**: Incorporar una nueva fuente de datos, como el análisis de sentimiento de noticias de finanzas en español (ej. de El Financiero, Expansión) para añadir una característica que capture el "mood" del mercado local.
3.  **Backtesting Riguroso**: Implementar una estrategia de backtesting que simule operaciones de compra/venta basadas en las señales del modelo a lo largo de un período histórico para evaluar el rendimiento financiero real de la estrategia en el contexto de la BMV.
4.  **Orquestación y Automatización**: Utilizar herramientas como Airflow o Prefect para orquestar la ejecución diaria/semanal de este pipeline, automatizando la descarga de datos, el reentrenamiento, la optimización y el guardado de los nuevos artefactos del mejor modelo para el mercado mexicano.